In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

data = pd.read_excel('nba_player_data.xlsx')

: 

In [ ]:
data.sample(10) #顯示data frame中十個隨機樣本

# Data cleaning & analysis

In [3]:
data.drop(columns =['RANK', 'EFF'], inplace = True) #去掉「排名」和「效率」欄（不重要
data['season_start_year'] = data['Year'].str[:4].astype(int)

In [4]:
data['TEAM'].replace(to_replace=['NOP', 'NOH'], value='NO',inplace=True) 
#Team 'NOP' & 'NOH'are the same team (changed in 2013), so we merge them into 'NO'

In [5]:
data['Season_type'].replace('Regular%20Season', 'RS', inplace=True)
# change the index to RS --> 例行賽

In [6]:
rs_df = data[data['Season_type']=='RS']
playoffs_df = data[data['Season_type']=='Playoffs']

In [ ]:
data.columns

In [8]:
total_cols = ['MIN','FGM','FGA','FG3M','FG3A','FTM','FTA',
              'OREB','DREB','REB','AST','STL','BLK','TOV','PF','PTS']
#篩選我要的數據

# Which player stat are correlated with each other

In [ ]:
#以球員為單位，整理這十年在total_cols欄位裡的數據
data_per_min = data.groupby(['PLAYER','PLAYER_ID','Year'])[total_cols].sum().reset_index()
#我們將收集的是總數據-->除以「總比賽時數」--> made/attempt -->整理出「進階數據」
for col in data_per_min.columns[4:]:
    data_per_min[col] = data_per_min[col]/data_per_min['MIN']
#Advance stat
data_per_min['FG%'] = data_per_min['FGM']/data_per_min['FGA']      #field goal percentage
data_per_min['3PT%'] = data_per_min['FG3M']/data_per_min['FG3A']   #3-point field goal percentage
data_per_min['FT%'] = data_per_min['FTM']/data_per_min['FTA']      #free throw percentage
data_per_min['FG3A%'] = data_per_min['FG3A']/data_per_min['FGA']   #3-point attempt percentage
data_per_min['PTS/FGA'] = data_per_min['PTS']/data_per_min['FGA']  #scoring efficiency
data_per_min['FG3M/FGM'] = data_per_min['FG3M']/data_per_min['FGM'] #3-point efficiency
data_per_min['FTA/FGA'] = data_per_min['FTA']/data_per_min['FGA']   #free-throw attempt per field goal attempt
data_per_min['TRU%'] = 0.5*data_per_min['PTS']/(data_per_min['FGA']+0.475*data_per_min['FTA']) #true-shooting percentage
data_per_min['AST_TOV'] = data_per_min['AST']/data_per_min['TOV'] #Turnover ratio

#篩掉球員賽季總比賽時數<50分鐘
data_per_min = data_per_min[data_per_min['MIN']>=50]
#不需要做player id的相關性
data_per_min.drop(columns='PLAYER_ID', inplace=True)

#plot graph
fig = px.imshow(data_per_min.corr(numeric_only=True))
fig.show()

# How are minutes played distributed?

In [ ]:
#球員在「例行賽」的總分鐘數「百分比分佈」
fig = px.histogram(x=rs_df['MIN'], histnorm = 'percent')
#球員在「季候賽」的總分鐘數「百分比分佈」
fig_2 = px.histogram(x=playoffs_df['MIN'], histnorm = 'percent')

fig.update_layout(xaxis_title='total minutes played (per regular season)')
fig_2.update_layout(xaxis_title='total minutes played (per playoffs)')
fig.show()
fig_2.show()

In [ ]:
# 時數篩選function
def hist_data(df=rs_df, min_MIN=0, min_GP=0):
    return df.loc[(df['MIN']>=min_MIN) & (df['GP']>=min_GP), 'MIN']/\
    df.loc[(df['MIN']>=min_MIN) & (df['GP']>=min_GP), 'GP']

fig = go.Figure()
fig.add_trace(go.Histogram(x = hist_data(rs_df, 50 ,5), histnorm='percent', name='Regular Season',
                          xbins = {'start':0, 'end':46, 'size':1}))
fig.add_trace(go.Histogram(x = hist_data(playoffs_df, 5, 1), histnorm='percent', 
                           name='Playoffs',xbins = {'start':0, 'end':46, 'size':1}))

fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.5)
fig.show()

In [12]:
rs_12to34_min_percentage = ((hist_data(rs_df,50,5)>=12)&(hist_data(rs_df,50,5)<=34)).mean()

In [13]:
playoffs_12to34_min_percentage = ((hist_data(playoffs_df,5,1)>=12)&(hist_data(playoffs_df,5,1)<=34)).mean()

# How has the game changed over the past 10 years?

In [ ]:
change_df = data.groupby('season_start_year')[total_cols].sum().reset_index() #依季度之「數據總和」表
#Possesion（每一波攻勢） estimation
change_df['POSS_est'] = change_df['FGA']-change_df['OREB']+change_df['TOV']+0.44*change_df['FTA']
#將POSS_est欄位移到第三左column
change_df = change_df[list(change_df.columns[0:2])+['POSS_est']+list(change_df.columns[2:-1])]

change_df['FG%'] = change_df['FGM']/change_df['FGA']
change_df['3PT%'] = change_df['FG3M']/change_df['FG3A']
change_df['FT%'] = change_df['FTM']/change_df['FTA']
change_df['AST%'] = change_df['AST']/change_df['FGM']
change_df['FG3A%'] = change_df['FG3A']/change_df['FGA']
change_df['PTS/FGA'] = change_df['PTS']/change_df['FGA']
change_df['FG3M/FGM'] = change_df['FG3M']/change_df['FGM']
change_df['FTA/FGA'] = change_df['FTA']/change_df['FGA']
change_df['TRU%'] = 0.5*change_df['PTS']/(change_df['FGA']+0.475*change_df['FTA'])
change_df['AST_TOV'] = change_df['AST']/change_df['TOV']

change_df

In [ ]:
change_per48_df = change_df.copy() #per 48 minutes (一場比賽的總時長)
for col in change_per48_df.columns[2:18]:
    change_per48_df[col] = (change_per48_df[col]/change_per48_df['MIN'])*48*5 
    #average stat per minutes per team（乘上場上五人)
    
change_per48_df.drop(columns = 'MIN', inplace=True)

fig = go.Figure()
for col in change_per48_df.columns[1:]:
    fig.add_trace(go.Scatter(x=change_per48_df['season_start_year'],
                             y=change_per48_df[col], name=col))
fig.show()

In [ ]:
change_per100_df = change_df.copy()

for col in change_per100_df.columns[3:18]:
    change_per100_df[col] = (change_per100_df[col]/change_per100_df['POSS_est'])*100
     #average stat per 100 possesion
        
change_per100_df.drop(columns=['MIN','POSS_est'], inplace=True)

fig = go.Figure()
for col in change_per100_df.columns[1:]:
    fig.add_trace(go.Scatter(x=change_per100_df['season_start_year'],
                             y=change_per100_df[col], name=col))
fig.show()

In [ ]:
## Compare Regular season to Playoffs

: 

In [ ]:
rs_change_df = rs_df.groupby('season_start_year')[total_cols].sum().reset_index()
playoffs_change_df = playoffs_df.groupby('season_start_year')[total_cols].sum().reset_index()

for i in [rs_change_df,playoffs_change_df]:
    i['POSS_est'] = i['FGA']-i['OREB']+i['TOV']+0.44*i['FTA']
    i['POSS_per_48'] = (i['POSS_est']/i['MIN'])*48*5
    #POSS_per_48 --> 一場比賽一隊平均有幾個攻勢
    i['FG%'] = i['FGM']/i['FGA']
    i['3PT%'] = i['FG3M']/i['FG3A']
    i['FT%'] = i['FTM']/i['FTA']
    i['AST%'] = i['AST']/i['FGM']
    i['FG3A%'] = i['FG3A']/i['FGA']
    i['PTS/FGA'] = i['PTS']/i['FGA']
    i['FG3M/FGM'] = i['FG3M']/i['FGM']
    i['FTA/FGA'] = i['FTA']/i['FGA']
    i['TRU%'] = 0.5*i['PTS']/(i['FGA']+0.475*i['FTA'])
    i['AST_TOV'] = i['AST']/i['TOV']
    
    for col in total_cols:
        i[col] = 100*i[col]/i['POSS_est']
        #換成average stat per 100 persession
    i.drop(columns=['MIN','POSS_est'], inplace=True)
    
rs_change_df

In [ ]:
playoffs_change_df

In [ ]:
#「季後賽」對「例行賽」的「數據變化」百分差
comp_change_df = round(100*(playoffs_change_df-rs_change_df)/rs_change_df,3)
comp_change_df['season_start_year'] = list(range(2012,2022))
comp_change_df

In [ ]:
fig = go.Figure()
for col in comp_change_df.columns[1:]:
    fig.add_trace(go.Scatter(x=comp_change_df['season_start_year'],
                             y=comp_change_df[col], name=col))
fig.show()